# End-to-End NLP: News Headline Classifier (SageMaker Version)

_**Train a Keras-based model to classify news headlines between four domains**_

This notebook works well with the `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)` kernel on SageMaker Studio, or `conda_tensorflow2_latest_p37` on classic SageMaker Notebook Instances.


---

Following on from the previous 'local version' notebook, we show here how to trigger the model training and deployment on separate infrastructure - to make better use of resources.

Note that you can safely ignore the WARNING about the pip version.

In [ ]:
# First install some libraries which might not be available across all kernels (e.g. in Studio):
!pip install ipywidgets

### Set Up Execution Role and Session

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [ ]:
%%time
%load_ext autoreload
%autoreload 2

import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()


### Download News Aggregator Dataset

We will download **FastAi AG News** dataset from the https://registry.opendata.aws/fast-ai-nlp/ public repository. This dataset contains a table of news headlines and their corresponding classes.


In [ ]:
%%time
import util.preprocessing

util.preprocessing.download_dataset()


### Let's visualize the dataset

We will load the ag_news_csv/train.csv file to a Pandas dataframe for our data processing work.

In [ ]:
import os
import re
import numpy as np
import pandas as pd


In [ ]:
column_names = ["CATEGORY", "TITLE", "CONTENT"]
# we use the train.csv only
df = pd.read_csv("data/ag_news_csv/train.csv", names=column_names, header=None, delimiter=",")
# shuffle the DataFrame rows
df = df.sample(frac=1, random_state=1337)
# make the category classes more readable
mapping = {1: 'World', 2: 'Sports', 3: 'Business', 4: 'Sci/Tech'}
df = df.replace({'CATEGORY': mapping})
df.head()


For this exercise we'll **only use**:

- The **title** (Headline) of the news story, as our input
- The **category**, as our target variable


In [ ]:
df["CATEGORY"].value_counts()


The dataset has **four article categories** with equal weighting:

- Business
- Sci/Tech
- Sports
- World


## Natural Language Pre-Processing

We'll do some basic processing of the text data to convert it into numerical form that the algorithm will be able to consume to create a model.

We will do typical pre processing for NLP workloads such as: dummy encoding the labels, tokenizing the documents and set fixed sequence lengths for input feature dimension, padding documents to have fixed length input vectors.


### Dummy Encode the Labels


In [ ]:
encoded_y, labels = util.preprocessing.dummy_encode_labels(df, "CATEGORY")
print(labels)


In [ ]:
df["CATEGORY"].iloc[0]

In [ ]:
encoded_y[0]

### Tokenize and Set Fixed Sequence Lengths

We want to describe our inputs at the more meaningful word level (rather than individual characters), and ensure a fixed length of the input feature dimension.


In [ ]:
padded_docs, tokenizer = util.preprocessing.tokenize_and_pad_docs(df, "TITLE")


In [ ]:
df["TITLE"].iloc[0]

In [ ]:
padded_docs[0]

### Import Word Embeddings

To represent our words in numeric form, we'll use pre-trained vector representations for each word in the vocabulary: In this case we'll be using [pre-trained word embeddings from FastText](https://fasttext.cc/docs/en/crawl-vectors.html), which are also available for a broad range of languages other than English. Be mindful if you are encounterring file formatting error, you may consider deleting fastText word embedding file before re-running `get_word_embeddings` function.

You could also explore training custom, domain-specific word embeddings using SageMaker's built-in [BlazingText algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html). See the official [blazingtext_word2vec_text8 sample](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/blazingtext_word2vec_text8) for an example notebook showing how.


In [ ]:
%%time
embedding_matrix = util.preprocessing.get_word_embeddings(tokenizer, "data/embeddings")


In [ ]:
np.save(
    file="./data/embeddings/docs-embedding-matrix",
    arr=embedding_matrix,
    allow_pickle=False,
)
vocab_size=embedding_matrix.shape[0]
print(embedding_matrix.shape)


### Split Train and Test Sets

Finally we need to divide our data into model training and evaluation sets:


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    padded_docs,
    encoded_y,
    test_size=0.2,
    random_state=42
)


In [ ]:
os.makedirs("./data/train", exist_ok=True)
np.save("./data/train/train_X.npy", X_train)
np.save("./data/train/train_Y.npy", y_train)
os.makedirs("./data/test", exist_ok=True)
np.save("./data/test/test_X.npy", X_test)
np.save("./data/test/test_Y.npy", y_test)


## Upload Data to S3

We'll need to upload our processed data to S3 to make it available for SageMaker training jobs:


In [ ]:
s3_bucket = sess.default_bucket()
s3_prefix = "news"


In [ ]:
traindata_s3_prefix = f"{s3_prefix}/data/train"
testdata_s3_prefix = f"{s3_prefix}/data/test"
embeddings_s3_prefix = f"{s3_prefix}/data/embeddings"
output_s3 = f"s3://{s3_bucket}/{s3_prefix}/models/"


In [ ]:
train_s3 = sess.upload_data(path="./data/train/", bucket=s3_bucket, key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path="./data/test/", bucket=s3_bucket, key_prefix=testdata_s3_prefix)
embeddings_s3 = sess.upload_data(
    # Only send the numpy array of embeddings, not the original files as well:
    path="./data/embeddings/docs-embedding-matrix.npy",
    bucket=s3_bucket,
    key_prefix=embeddings_s3_prefix,
)


In [ ]:
inputs = { "train": train_s3, "test": test_s3, "embeddings": embeddings_s3 }
print(inputs)


## Train with Differentiated Infrastructure on Sagemaker

This time, we've packaged the model build and train code from our previous notebook ([Headline Classifier Local.ipynb](Headline%20Classifier%20Local.ipynb)) into the [**main.py**](src/main.py) script in the **src** directory.

We'll use the high-level [TensorFlow Framework Container through the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html) to train and deploy the model from this script file.

### How Amazon SageMaker runs your Tensorflow script with pre-built containers

AWS provides a pre-packaged set of Docker images to help you accelerate building your projects on major ML frameworks: The [SageMaker Framework Containers](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-prebuilt.html).

These containers take care of basic setup like GPU drivers, serving stack implementation, core libraries, and so on - leaving us free to simply inject some Python scripts for the training process and any inference behaviour overrides. We can even provide a *requirements.txt* file to specify additional dependencies to be dynamically installed at start-up - without having to build these into the container image.

**As a result, our first task is to understand the interfaces** between our script(s) and the runtime: How will the script read input data? Parameters? Where should it store results?

#### Running your container during training

When Amazon SageMaker runs training, your training script (entry_point input) is run just like a regular Python program. A number of files are laid out for your use, under a `/opt/ml` directory. These will be locations that you can access from within your script. You will see an example of the use of this in our [**main.py**](src/main.py):

    /opt/ml
    |-- code
    |   `-- <our script(s)>
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure

##### The input

* `/opt/ml/input/config` contains information to control how your program runs. `hyperparameters.json` is a JSON-formatted dictionary of hyperparameter names to values. These values will always be strings, so you may need to convert them. `resourceConfig.json` is a JSON-formatted file that describes the network layout used for distributed training. Since scikit-learn doesn't support distributed training, we'll ignore it here.
* `/opt/ml/input/data/<channel_name>/` (for File mode) contains the input data for that channel. The channels are created based on the call to CreateTrainingJob but it's generally important that channels match what the algorithm expects. The files for each channel will be copied from S3 to this directory, preserving the tree structure indicated by the S3 key structure. 
* `/opt/ml/input/data/<channel_name>_<epoch_number>` (for Pipe mode) is the pipe for a given epoch. Epochs start at zero and go up by one each time you read them. There is no limit to the number of epochs that you can run, but you must close each pipe before reading the next epoch.

##### The output

* `/opt/ml/model/` is the directory where you write the model that your algorithm generates. Your model can be in any format that you want. It can be a single file or a whole directory tree. SageMaker will package any files in this directory into a compressed tar archive file. This file will be available at the S3 location returned in the `DescribeTrainingJob` result.
* `/opt/ml/output` is a directory where the algorithm can write a file `failure` that describes why the job failed. The contents of this file will be returned in the `FailureReason` field of the `DescribeTrainingJob` result. For jobs that succeed, there is no reason to write this file as it will be ignored.

#### Further information

For more information, you can refer to:

- The [SageMaker Python SDK guide for TensorFlow](https://sagemaker.readthedocs.io/en/stable/using_tf.html) and [API doc](https://sagemaker.readthedocs.io/en/stable/sagemaker.tensorflow.html) for TensorFlow framework classes.
- The [AWS Deep Learning Containers repository](https://github.com/aws/deep-learning-containers) on GitHub, which defines the underlying container images.
- The open source SageMaker [TensorFlow Training Toolkit](https://github.com/aws/sagemaker-tensorflow-training-toolkit) and [TensorFlow Serving Container](https://github.com/aws/sagemaker-tensorflow-serving-container) for more details on the framework code for training and serving.

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow as TensorFlowEstimator


Although the script will run on a separate container, we can pass whatever parameters it needs through SageMaker:


In [ ]:
hyperparameters = { "epochs": 5, "max_seq_len": 40, "num_classes": 4 }


We have our `TensorFlow` estimator object, we have set the hyper-parameters for this object and we have our data channels linked with the algorithm. The only  remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instance that we requested while creating the `TensorFlow` estimator classes is provisioned and is setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take some time, depending on the size of the data. 

Once the job has finished a "Job complete" message will be printed. The trained model can be found in the S3 bucket that was setup as `output_path` in the estimator.

We will run the training job using a `ml.g4dn.xlarge` instance (with modest GPU) to accelerate our training. If your account runs into resource limits please try a `ml.c5.xlarge` (CPU-only) or `ml.p2.xlarge` (alternative GPU) instance type. 

In [ ]:
metric_definitions = [
    { "Name": "loss", "Regex": "loss: ([0-9\\.]+)" },
    { "Name": "accuracy", "Regex": "acc: ([0-9\\.]+)" },
    { "Name": "validation:loss", "Regex": "Validation.*loss=([0-9\\.]+)" },
    { "Name": "validation:accuracy", "Regex": "Validation.*acc=([0-9\\.]+)" },
]

estimator = TensorFlowEstimator(
    framework_version="2.4",
    py_version="py37",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    role=role,
    entry_point="main.py",
    source_dir="./src",
    distribution={ "parameter_server": { "enabled": True } },
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    base_job_name="news-keras",
    max_run=20*60,  # Maximum allowed active runtime
    use_spot_instances=True,  # Use spot instances to reduce cost
    max_wait=30*60,  # Maximum clock time (including spot delays)
)

estimator.fit(inputs)


While the training job is running take a minute to look at the `main.py` script. You can see how we have adapted the our original local code from [Headline Classifier Local.ipynb](Headline%20Classifier%20Local.ipynb) to run on Sagemaker.

## Use the Model: Hosting / Inference
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same type of instance that we used to train. Because instance endpoints will be up and running for long, it's advisable to choose a cheaper instance for inference.


In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
)


### Your model should now be in production as a RESTful API!

Let's evaluate our model with some example headlines...

If you struggle with the widget, you can always simply call the `classify()` function from Python.

You can be creative with your headlines!


In [ ]:
from IPython import display
import ipywidgets as widgets
from tensorflow.keras.preprocessing.sequence import pad_sequences

def classify(text):
    """Classify a headline and print the results"""
    encoded_example = tokenizer.texts_to_sequences([text])
    # Pad documents to a max length of 40 words
    max_length = 40
    padded_example = pad_sequences(encoded_example, maxlen=max_length, padding="post")
    result = predictor.predict(padded_example.tolist())
    print(result)
    ix = np.argmax(result["predictions"])
    print(f"Predicted class: '{labels[ix]}' with confidence {result['predictions'][0][ix]:.2%}")

interaction = widgets.interact_manual(
    classify,
    text=widgets.Text(
        value="The markets were bullish after news of the merger",
        placeholder="Type a news headline...",
        description="Headline:",
        layout=widgets.Layout(width="99%"),
    )
)
interaction.widget.children[1].description = "Classify!"


## Clean up

Unlike training jobs (which destroy their resources as soon as training is finished), real-time endpoint deployments provision instances until we specifically shut the endpoint down...

So let's be frugal with resources, and delete resources when we don't need them anymore:


In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)

## (Optional) Automatic Hyperparameter Optimization - HPO

Rather than manually tweak parameters to tune the model performance, we can get SageMaker to help us out.

We'll simply tell SageMaker:

- The type and allowable range of each parameter,
- The metric we want to optimize for, and
- Strategy and resource constraints

...and the service will set up jobs for us to find the best combination.


### (Hyper-)Parameter Definitions


In [ ]:
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner, IntegerParameter

hyperparameter_ranges = {
    "epochs": IntegerParameter(2, 7),
    "learning_rate": ContinuousParameter(0.01, 0.2),
}


### Objective Metric

'Metrics' in SageMaker are scraped from the console output of jobs, by way of regular expressions.

We can define multiple metrics to monitor, but HPO requires us to specify that exactly one of them is the **objective** metric to optimize:


In [ ]:
# (See metric_definitions above)
objective_metric_name = "validation:accuracy"
objective_type = "Maximize"


### Start the Tuning Job

We already defined our Estimator above, so we'll just re-use the configuration with minor adjustments.

Note that the Estimator's `hyperparameters` will be used as base values, and overridden by the HyperParameterTuner where appropriate.


In [ ]:
# Keep per-job resources modest, so that parallel jobs don't hit any limits:
estimator.instance_type = "ml.c5.xlarge"
estimator.instance_count = 1

In [ ]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    base_tuning_job_name="news-hpo-keras",
    max_jobs=6,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

tuner.fit(inputs)


### Check On Progress

HPO jobs can take a long time to complete, and can run multiple training jobs in parallel - each on multiple instances... Which is why the `fit()` call above won't show us a potentially-confusing consolidated log stream, and may not wait for completion if we add the `wait=False` parameter.

Go to the Training > Hyperparameter Tuning Jobs page of the [**SageMaker Console**](https://console.aws.amazon.com/sagemaker/home#/hyper-tuning-jobs) and select the job from the list.

You can see all the training jobs triggered for the HPO run, as well as overall summary metrics.

This information can be accessed via the API/SDKs too of course. For example we can wait for HPO to finish like the below:


In [ ]:
import boto3
import time

# Wait until HPO is finished
hpo_state = None
smclient = boto3.Session().client("sagemaker")

while hpo_state is None or hpo_state == "InProgress":
    if hpo_state is not None:
        print("-", end="")
        time.sleep(60)  # Poll once every 1 min
    hpo_state = smclient.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
    )["HyperParameterTuningJobStatus"]

print("\nHPO state:", hpo_state)


### Using the Model

Just like with our `estimator`, we can call `tuner.deploy()` to create an endpoint and `predictor` from the best-performing model found in the HPO run.


## Review

In this notebook, we refactored our local code to train and deploy the same Keras model using SageMaker.

Some benefits of this approach are:

- We can automatically provision specialist computing resources (e.g. high-performance, or GPU-accelerated instances) for **only** the duration of the training job: Getting good performance in training, without leaving resources sitting around under-utilized
- The history of training jobs (including parameters, metrics, outputs, etc.) is automatically tracked - unlike local notebook experiments where the user needs to keep notes on what worked and what didn't
- Our trained model can be deployed to a secure, production-ready web endpoint with just one SDK call: No container or web application packaging required, unless we want to customize the behaviour
